<a href="https://colab.research.google.com/github/sowmya-520/AI/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                                Assignment 6
Estimate the bicarbonates of well water based on its pH value using polynomial regression model. Consider SGD optimizer.

Do the manual calculation for one iteration by taking only first two samples in the dataset
* Write the python code to build simple linear regression model using SGD optimizer
*Do the data normalization
* Split the data for train and test (90:10)
* Train the simple linear regression model using SGD with training data
* compute MSE, RMSE and MAE with training data and testing data

**Mount Google Drive**

In [207]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [208]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.stats import zscore
import math as mt

**Load Data**

In [209]:
# loading the data from csv file to pandas dataframe
data = pd.read_csv('/content/drive/MyDrive/slr09 (2).csv')
#Displaying the first five rows in table
data.head()

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171


**Outliers Detection**

In [210]:
outliers = []
def outliers_detection(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers

In [211]:
sample_outliers_X = detect_outliers_zscore(data['X'])
sample_outliers_Y = detect_outliers_zscore(data['Y'])
print("Outliers in X column from Z-scores method: ", sample_outliers_X)
print("Outliers in Y column from Z-scores method: ", sample_outliers_Y)

Outliers in X column from Z-scores method:  []
Outliers in Y column from Z-scores method:  []


**Outlier Treatement**

In [212]:
zX=np.abs(zscore(data['X']))
zY=np.abs(zscore(data['Y'])) 
data['z_score_X']=zX 
data['z_score_Y']=zY
outliersX=len(data[data['z_score_X']>3]) 
outliersY=len(data[data['z_score_Y']>3])
data_z = data[data['z_score_X']<3] 
data=data_z.drop(['z_score_X', 'z_score_Y'], axis=1) 
data

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171
5,7.8,143
6,7.3,217
7,8.0,190
8,7.1,142
9,7.5,190


**Missing Values Treatment**

In [213]:
print(data.isnull().sum())

X    0
Y    0
dtype: int64


In [214]:
m1=data['X'].mean()
m2=data['Y'].mean()
data['X'].fillna(value=m1, inplace=True)
data['Y'].fillna(value=m2, inplace=True)
data

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171
5,7.8,143
6,7.3,217
7,8.0,190
8,7.1,142
9,7.5,190


**Split Data**

---


> 90%-->training
    10%-->Testing


In [215]:
x_train, x_test, y_train, y_test = train_test_split(data.X, data.Y, test_size = 0.1, random_state = 10)

**Intialisation**

In [216]:
m1=random.uniform(-5,5)
m2=random.uniform(-5,5)
c=random.uniform(-5,5)
eta=0.0001
max_iter=10

**Gradient Calculation**

In [217]:
def gradient(m1,m2,c,x,y):
  m1_deriv=-1*(y-m1*x*x-m2*x-c)*x*x
  m2_deriv=-1*(y-m1*x*x-m2*x-c)*x
  c_deriv=-1*(y-m1*x*x-m2*x-c)
  return m1_deriv,m2_deriv,c_deriv

**Training Using Stochastic Gradient Descent Optimizer**

In [218]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_c=gradient(m1, m2, c, x_train.iloc[sample], y_train.iloc[sample])
    m1=m1-(eta)*del_m1
    m2=m2-(eta)*del_m2
    c=c-(eta)*del_c
print('model parameter m1, m2 and c',m1,m2,c)

model parameter m1, m2 and c 2.702721932267279 -1.2740337399392307 -4.839972934304336


**Model Prediction on Training Data**

In [219]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m1 * i*i)+(m2 * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

**Model Prediction on Testing Data**

In [220]:
y_pred_test = []
for i in x_test:
  y_p_te = (m1 * i*i)+(m2 * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

**Error Metrics Calculation**

In [221]:
#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 66.0541760716565
Mean square error 4363.154176505397
Mean absolute error 52.90976846495025


In [222]:
#Testing Accuracies
rmse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', rmse)
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse)  
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Mean square error 2825.8976902036666
Root mean square error 53.159173151993876
Mean absolute error 37.07250331774357


**Save Model For Deployment**

In [223]:
i=(input('enter pH Level : '))
Bicarbonate=m1*float(i)*float(i)+m2*float(i)+c
print('pH of well water : ',float(i))
print('Bicarbonate (parts per million) of well water: ',Bicarbonate)

enter pH Level : 5.6
pH of well water :  5.6
Bicarbonate (parts per million) of well water:  72.78279791793781
